Pro. Fernando Amaral  -  [www.eia.ai](https://www.eia.ai)

In [1]:
#gpu
!pip install transformers transformers[torch] datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


Fine Tunning

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset
import torch

In [3]:
model_name = "pierreguillou/gpt2-small-portuguese"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/92.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/508k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [4]:
# dados para fine tunning
import requests
from datasets import Dataset

url = "https://pt.wikipedia.org/wiki/Ci%C3%AAncia"
response = requests.get(url)
text_data = response.text

lines = text_data.split('\n')
data_dict = {"text": lines}

dataset = Dataset.from_dict(data_dict)

In [5]:
# tokenização do dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/2086 [00:00<?, ? examples/s]

In [17]:
first_example = tokenized_dataset[1349]
print(first_example)

{'text': '<p>As atitudes <a href="/wiki/Antici%C3%AAncia" title="Anticiência">anticientíficas</a> parecem ser frequentemente causadas pelo medo da rejeição nos grupos sociais. Por exemplo, as alterações climáticas são percebidas como uma ameaça por apenas 22% dos da população estadunidense no <a href="/wiki/Direita_(pol%C3%ADtica)" title="Direita (política)">lado direito</a> do <a href="/wiki/Espectro_pol%C3%ADtico" title="Espectro político">espectro político</a>, mas por 85% dos que estão do <a href="/wiki/Esquerda_(pol%C3%ADtica)" title="Esquerda (política)">lado esquerdo</a>.<sup id="cite_ref-256" class="reference"><a href="#cite_note-256"><span>[</span>255<span>]</span></a></sup> Ou seja, se alguém da direita considerar as alterações climáticas como uma ameaça, essa pessoa poderá enfrentar o desprezo e ser rejeitada naquele grupo social. Na verdade, as pessoas podem preferir negar um fato cientificamente aceito do que perder ou pôr em risco o seu estatuto social.<sup id="cite_ref-2

In [18]:
data_collator = DataCollatorForLanguageModeling( tokenizer=tokenizer,mlm=False)

In [19]:
# hiper parâmetros
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=1, # !!!
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2
)

In [20]:
trainer = Trainer(model=model,args=training_args, data_collator=data_collator, train_dataset=tokenized_dataset)

In [21]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=66, training_loss=2.503112099387429, metrics={'train_runtime': 77.727, 'train_samples_per_second': 26.838, 'train_steps_per_second': 0.849, 'total_flos': 136263794688000.0, 'train_loss': 2.503112099387429, 'epoch': 1.0})

In [22]:
# Salvar
model.save_pretrained("meu_modelo_treinado")
tokenizer.save_pretrained("meu_modelo_treinado")

('meu_modelo_treinado/tokenizer_config.json',
 'meu_modelo_treinado/special_tokens_map.json',
 'meu_modelo_treinado/vocab.json',
 'meu_modelo_treinado/merges.txt',
 'meu_modelo_treinado/added_tokens.json')

Usando o Modelo

In [23]:
from transformers import pipeline

In [26]:
gerador = pipeline("text-generation", model="meu_modelo_treinado")
texto = "Em sentido estrito, ciência refere-se ao sistema de adquirir conhecimento baseado no método científico."
resultado = gerador(texto, max_length=128, do_sample=True, temperature=0.7, top_k=50, top_p=0.85)
print(resultado)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': 'Em sentido estrito, ciência refere-se ao sistema de adquirir conhecimento baseado no método científico. O conhecimento científico, por sua vez, refere-se ao sistema de conhecimento obtido de forma sistemática.\n\nA ciência está ligada à ciência da ciência. O estudo científico, por sua vez, está ligada à ciência da ciência da ciência da ciência. O estudo científico, por sua vez, está ligado à ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência'}]


In [27]:
print(resultado[0]['generated_text'])

Em sentido estrito, ciência refere-se ao sistema de adquirir conhecimento baseado no método científico. O conhecimento científico, por sua vez, refere-se ao sistema de conhecimento obtido de forma sistemática.

A ciência está ligada à ciência da ciência. O estudo científico, por sua vez, está ligada à ciência da ciência da ciência da ciência. O estudo científico, por sua vez, está ligado à ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência da ciência
